In [4]:
!pwd

/home/sridiyer/raw_data


In [3]:
import pandas as pd
import gzip
import json
import pickle

In [17]:
# process product meta data and store it in a list of python dictionary
# we can use the dictionary for loading direct into weaviate
# (i.e., no need to use pandas dataframe - covert to dictionary later)

prod_meta_data_list = [] # list of dictionaries (each dict - product meta data)

with gzip.open('meta_Electronics.json.gz') as f:
    for l in f:
        # load each line in the input file
        
        row = json.loads(l.strip())
        
        # pre-processing getting rid of rows with bad data
        if row['description'] ==[]:
            continue
        if (row['category'] =="") or (row['category'] ==[]):
            continue
        if row['feature'] == []:
            continue
        if (row['title'] =="") or (row['title'] ==[]):
            continue
        if not isinstance (row['asin'], str):
            continue
        # for price strip $ is any. there may still be
        # problems because people may have put comma instead of
        # period in price $ 2,40 instead of just 2.40
        try:
            rprice = float(row['price'].replace("$","").replace(",","."))
        except ValueError:
            continue
        
        # since we are storing data in picke format, we can also store
        # processed data in dictionary format (instead of pandas dataframe
        # this helps with loading into weaviate
        
        prod_meta_dict = {
            'asin' : row['asin'],
            'title' : row ['title'],
            'description' : row ['description'][0].replace("\n", " ").replace("\t", " "),
            'category' : row ['category'],
            'mainCategory' : row['main_cat'],
            'brand' : row ['brand'],
            'feature' :  row ['feature'],
            'price' : rprice
          }    
        prod_meta_data_list.append(prod_meta_dict)

print (' num products in the processed data  ', len(prod_meta_data_list))
    

 num products in the processed data   266050


In [18]:
# save data in a picke file

prod_meta_fn = 'proc_prod_meta_data_may22.pkl'

with open(prod_meta_fn, 'wb') as fhandle:
    pickle.dump(prod_meta_data_list, fhandle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
prod_review_list = []

with gzip.open('Electronics.json.gz') as fp:
    for lp in fp:
        row = json.loads(lp.strip())

        if ('summary' not in row.keys()) or ('reviewText' not in row.keys()):
            continue 
        if (row['reviewText'] =="") or (row['reviewText'] ==" "):
            continue
        if (row['summary'] =="") or (row['summary'] ==" "):
            continue
        if (row['reviewerID'] =="") or (row['reviewerID'] ==" "):
            continue
        if not isinstance (row['asin'], str):
            continue
        try:
            r_overall = float(row['overall'])
        except ValueError:
            continue
            
        review_object = {
            'reviewText' : row['reviewText'].replace("\n", " ").replace("\t"," "),
            'summary' : row ['summary'],
            'overall' : r_overall,
            'reviewerid' : row ['reviewerID'],
            'reviwerName' : row.get('reviewerName', 'GENERIC'),
            'productasin' : row['asin']
          }  
        
        prod_review_list.append(review_object)
              
print (' num reviews in the processed data  ', len(prod_review_list))
 


 num reviews in the processed data   20980243


In [6]:
import _pickle as cPickle

In [8]:
import _pickle as pickle
#import cPickle as pickle
prod_review_fn = 'proc_review_data_may22.pkl'
p = pickle.Pickler(open(prod_review_fn,"wb")) 
p.fast = True 
p.dump(prod_review_list) 

In [2]:

prod_review_fn = 'proc_review_data_may22.pkl'

with open(prod_review_fn, 'wb') as fh:
    pickle.dump(prod_review_list, fh)

NameError: name 'prod_review_list' is not defined